# Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

# Sequence-to-Sequence (seq2seq)

paper: https://arxiv.org/pdf/1409.3215

## Introduction

기존 모델의 한계: 입력과 출력이 고정된 차원의 벡터로 인코딩될 수 있는 문제에만 적용 가능 <br>
-> 음성 인식과 기계 번역 및 질문 응답 등은 sequence 문제이며, sequence를 sequence로 mapping할 수 있는 도메인 독립적인 방법이 요구됨 <br>

<br>

Contribution: LSTM을 기반으로 sequence-to-sequence 문제 해결 방법 제안
- LSTM을 사용하여 input sequence를 encoding 후 고정 차원의 vector로 변환
- 또 다른 LSTM(decoder)을 통해 output sequence 생성

<br>

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*0hNJAH3xgW_6smcsz0vaWw.png)

## Model

LSTM은 $(x_1, ..., x_T)$가 입력될 때 조건부 확률 $p(y_1, ..., y_{T'}|x_1, ..., x_T)$ 생성 <br>
$y_1, ..., y_{T'}$은 입력 길이 T와 다를 수 있는 길이 $T'$을 갖는 input sequence에 mapping되는 output sequence <br>
LSTM은 고정된 vector representation인 벡터 $v$를 input seuqnce $(x_1, ..., x_T)$의 마지막 hidden state를 계산하기 위해 사용 <br>
이러한 $v$를 decoder의 hidden state의 initial 값으로 설정하여 $y_1, ..., y_T$의 확률 계산

$$
p(y_1, ..., y_{T'}|x_1, ..., x_T) = \Pi_{t=1}^{T'}p(y_t|v, y_1, ..., y_{t-1})
$$

위의 식에서 각 $p(y_t|v, y_1, ..., y_{t-1})$ 분포는 vocabuary 내 전체 단어에 대한 softmax로 표현 <br>
각 sequence는 special symbol인 \<EOS\>로 끝나는데, 이는 모든 가능한 길이의 문장에 대한 분포를 정의할 수 있게 함 <br>
즉, 위의 figure에서 Encoder 입력은 'A', 'B', 'C', '\<EOS\>'이며, Decoder 입력은 'W', 'X', 'W', 'Y', 'Z', '\<EOS\>'가 됨 <br>

<br>

<span style='font-size: 18px'> 모델 구현 </span>
1. 두 개의 다른 LSTM 사용
    - input sequence에 대한 LSTM
    - output sequence에 대한 LSTM <br>
    -> 약간의 계산 비용(neglibible computational cost)을 증가시킴 <br>
    -> 서로 다른 언어 조합에 대한 LSTM을 동시에 학습시키는 것을 자연스럽게 함 <br>
2. deep LSTM이 shallow LSTM보다 유의하게 performance가 좋음
    - paper에서는 4개의 layer 사용
3. input sequence의 단어 순서를 뒤집는 것이 performance 향상에 큰 영향을 미침 <br>
    -> a, b, c를 $\alpha, \beta, \gamma$에 mapping하는것보다 c, b, a를 $\alpha, \beta, \gamma$에 mapping하는 것이 훨씬 유의 (a-$\alpha$, b-$\beta$, c-$\gamma$가 번역 쌍) <br>
    -> 이러한 방법으로, a가 $\alpha$, b가 $\beta$, c가 $\gamma$와 유사한 의미를 갖도록 할 수 있었음 <br>
&nbsp;&nbsp;&nbsp;&nbsp;        <- SGD(optimizer)가 입력과 출력 사이의 통신을(establish communication)을 쉽게 함

## Experiments

<span style='font-size: 18px'> 1. Dataset details </span>

source language에서 160k, target language에서 80k의 가장 자주 사용되는 용어를 사용 <br>
out-of-vocabulary word는 "UNK" token으로 대체 <br>

<br>


<span style='font-size: 18px'> 2. Decoding and Rescoring </span>

많은 sentence pair에 대해 매우 깊은 LSTM을 학습하여야 했기에, <br>
주어진 source sentence에 대해 올바른 번역의 log probability를 최대화하는 방향으로 학습 <br>

$$
1/|S| \Sigma_{(T,S) \in S} \log p(T|S)
$$,
where $S$: training set의 문장

<br>

학습이 종료되면, LSTM에서 가장 확률이 높은 번역을 찾아 번역 진행

$$
\hat{T} = \arg \max_{T} p(T|S)
$$

<br>

부분 가설이 일부 번역의 prefix를 가지는 소수의 부분 가설 B를 유지하는 left-to-right beam search decoder를 사용해 가장 높은 확률의 번역 탐색 <br>
> (We search for the most likely translation using a simple left-to-right beam search decoder which maintains a small number B of partial hypotheses, where a partial hypothesis is a prefix of some translation.)

- 부분 가설(partial hypothesis): 현재까지 생성된 번역의 일부분
- 빔 크기 (B): 유지할 부분 가설의 수. 값이 클수록 더 많은 후보군 고려
- left-to-right decoder: 번역을 왼쪽에서 오른쪽으로 생성

<br>

> <span style='font-size: 16px'> Example </span>
> 
> Hypothesis: <br>
> - input sequence: "I am happy"
> - target language: Korean
> - B: 2
> 
> <br>
> 
> 1.Initial Status
> - 디코더는 빈 문장에서 시작
> 
> 2.Step 1
> - 가능한 첫 단어 생성 (ex: 나는, 저는)
> - 각 단어의 초기 확률 계산 후, 가장 높은 확률의 상위 2개 후보 선택
>     - 상위 2개 후보: 부분가설
> 
> 3.Step 2
> - 부분 가설에서 다음 단어 생성
>     - "나는" 다음으로 가능한 단어와 그 확률 계산 (ex: 행복하다, 슬프다)
>     - "저는" 다음으로 가능한 단어와 그 확률 계산 (ex: 행복하다, 슬프다)
> - 각 부분 가설에 대해 새로운 후보군을 추가하고, 전체 확률을 기준으로 상위 2개 가설 선택
>     - 가설 1: 나는 행복하다   (확률: 0.6)
>     - 가설 2: 나는 슬프다    (확률: 0.5)
>     - 가설 3: 저는 행복하다   (확률: 0.4)
>     - 가설 4: 저는 슬프다    (확률: 0.3)
> 
> 4.Step 3
> - 위의 과정을 반복하며, 각 단계에서 최종 후보(가설)을 유지하며 문장 완성
> - Beam search는 가장 높은 확률의 전체 번역 선택
> 
> 각 단계에서 모든 단어 집합에 대해 beam의 각 가설 확대 <br>
> -> 가설 수가 크게 증가하기에 log probability에 따라 가장 가능성이 높은 B개의 가설을 제외하고 모두 폐기

<br>

최종적으로 "\<EOS\>" token이 가설에 추가되면, 이 token을 가설에서 제외하고 최종 가설의 집합에 추가

<br>

<span style='font-size: 18px'> 3. Reversing the Source Sentences </span>

저자는 dataset에 대한 다수의 단기 의존성 발생(introduction)으로 인한 것으로 추측 <br>
source와 target 문장 연결 시 source의 각 단어는 target의 단어와는 멀리 떨어져 있음 <br>
결과적으로 "minimal time lag" 문제를 가짐 <br>

<br>

source sentence의 단어를 뒤집는 것으로, source와 이와 대응대는 target의 평균 거리는 변하지 않음 <br>
그러나, source의 첫 몇 단어는 target의 첫 몇 문장과 매우 유사하여, large minimal time lag가 현격히 감소 <br>
이는 역전파에서 source와 target 사이의 더 쉬운 "establishing communication"을 갖도록 하여 전체적인 performance 향상에 유의하게 영향

<br>

> <span style='font-size: 16px'> Large minimal time lag </span>
> 
> input sequence와 oupput sequence 사이의 시간적 간격이 큰 것
>
> input sequence: I am learning French, which is my second language. <br>
> output sequence: J'apprends le français, qui est ma langue seconde. <br>
>
> 위의 경우 I am learning은 J'apprendre로 번역되는데 이 단어가 번역되기 까지의 시간이 오래 걸림

# Implementation

## Google

In [45]:
class Seq2Seq(nn.Module):
    def __init__(
            self,
            input_dim,
            latent_dim,
            vocab_size,
            n_layers: int = 4,
            teacher_forcing_ratio: float = 0.5,
        ):
        super().__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.vocab_size = vocab_size
        self.n_layers = n_layers
        self.teacher_forcing_ratio = teacher_forcing_ratio

        self.encoder = nn.LSTM(
            self.input_dim,
            self.latent_dim,
            num_layers=self.n_layers,
            batch_first=True,
        )

        self.decoder1 = nn.LSTM(
            self.latent_dim,
            128,
            batch_first=True,
            num_layers=4
        )
        self.decoder2 = nn.LSTM(
            128,
            128,
            batch_first=True,
            )
        self.decoder3 = nn.LSTM(
            128,
            128,
            batch_first=True,
            )
        self.decoder4 = nn.LSTM(
            128,
            1000,
            batch_first=True,
        )

    def forward(self, source, target):
        output, latent_vector = self.encoder(source)
        x = torch.zeros_like(output)[:, :1, :]
        output = self.decoder(x, latent_vector)
        next_token = output[:, 0, :].argmax(axis=-1)
        outputs = [next_token]
        for t in range(1, target.shape[1]):
            if np.random.random() > self.teacher_forcing_ratio:
                output = target[:, t, :]
            output, (h_n, c_n) = self.decoder(output, (h_n, c_n))

            next_token = output[:, 0, :].argmax(axis=-1)
            outputs.append(next_token)

        return outputs


In [ ]:
## 

In [224]:
# 인코더 클래스 정의
class Encoder(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, latent_dim: int, num_layers: int):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.latent_dim = latent_dim
        self.num_layers = num_layers

        # 단어 임베딩 레이어
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        # LSTM 레이어 정의
        self.lstm = nn.LSTM(
            self.embedding_dim,
            self.latent_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )
    
    def forward(self, x):
        # 입력을 임베딩
        x = self.embedding(x)
        # LSTM을 통해 출력 및 마지막 hidden state와 cell state 얻기
        x, (h_n, c_n) = self.lstm(x)

        # 마지막 hidden state와 cell state를 결합하여 잠재 벡터 생성
        latent_vector = torch.cat([h_n[-1], c_n[-1]], axis=-1)
        latent_vector = latent_vector[:, np.newaxis, :]  # 차원 추가

        return latent_vector


# 디코더 클래스 정의
class Decoder(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, latent_dim: int):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.latent_dim = latent_dim

        # 단어 임베딩 레이어
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        # 여러 개의 LSTM 레이어 정의
        self.lstm1 = nn.LSTM(self.embedding_dim + self.latent_dim * 2, self.latent_dim, batch_first=True)
        self.lstm2 = nn.LSTM(self.latent_dim, self.latent_dim, batch_first=True)
        self.lstm3 = nn.LSTM(self.latent_dim, self.latent_dim, batch_first=True)
        self.lstm4 = nn.LSTM(self.latent_dim, self.latent_dim, batch_first=True)
        # 최종 출력 레이어
        self.fc_out = nn.Linear(self.latent_dim, self.vocab_size)

    def forward(self, x, hidden_state, cell_state, latent_vector):
        x = x[:, np.newaxis]  # 차원 추가 (배치, 1)
        x = self.embedding(x)  # 입력 임베딩

        # 임베딩과 잠재 벡터 결합
        x = torch.cat([x, latent_vector], axis=-1)

        # 여러 LSTM 레이어를 순차적으로 통과
        x, _ = self.lstm1(x, (hidden_state, cell_state))
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, (h_n, c_n) = self.lstm4(x)  # 마지막 LSTM 레이어의 출력과 상태 반환
        x = self.fc_out(x)  # 최종 출력 생성 (단어 확률 분포)

        return x, (h_n, c_n)  # 출력 및 마지막 hidden state, cell state 반환


# Seq2Seq 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, teacher_forcing_ratio: float = 0.5):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.teacher_forcing_ratio = teacher_forcing_ratio  # teacher forcing 비율 설정
    
    def forward(self, source, target):
        batch_size = len(source)  # 배치 크기
        target_length = target.shape[1]  # 목표 시퀀스의 길이
        target_vocab_size = self.decoder.vocab_size  # 출력 어휘 크기
        outputs = torch.zeros(batch_size, target_length, target_vocab_size)  # 출력을 저장할 텐서 초기화

        # 인코더를 통해 잠재 벡터 생성
        latent_vector = self.encoder(source)
        x = target[:, 0]  # 첫 번째 디코더 입력 (<SOS> 토큰)
        h_n = torch.zeros(1, batch_size, self.encoder.latent_dim)  # 초기 hidden state
        c_n = torch.zeros(1, batch_size, self.encoder.latent_dim)  # 초기 cell state

        # 목표 시퀀스의 각 타임스텝에 대해 반복
        for t in range(1, target_length):
            # 디코더를 통해 출력 및 다음 hidden/cell state 얻기
            output, (h_n, c_n) = self.decoder(x, h_n, c_n, latent_vector)
            outputs[:, t - 1, :] = output[:, 0, :]  # 현재 타임스텝의 출력 저장

            # teacher forcing 사용 여부 결정
            if np.random.random() < self.teacher_forcing_ratio:
                x = output[:, 0, :].argmax(axis=-1)  # 모델의 출력을 다음 입력으로 사용
            else:
                x = target[:, t]  # 실제 타겟을 다음 입력으로 사용
        
        return outputs  # 모든 출력 반환

In [ ]:
# 예시 데이터 생성
source = torch.randint(0, 1000, (32, 20))   # 0 ~ 999의 정수 값을 갖는 (batch_size, seq_len) 행렬 생성
target = torch.randint(0, 1000, (32, 20))   # 0 ~ 999의 정수 값을 갖는 (batch_size, seq_len) 행렬 생성

In [228]:
# 인코더 초기화
encoder = Encoder(1000, 256, 256, 4)

# latent vector 생성
latent_vector = Encoder(1000, 256, 256, 4)(source)

In [229]:
hidden = torch.zeros(1, 32, 256)    # (num_layers, batch_size, latent_dim)
cell = torch.zeros(1, 32, 256)      # (num_layers, batch_size, latent_dim)

# 디코더 초기화
decoder = Decoder(1000, 256, 256)

# 디코더의 첫 번째 입력으로 target의 첫 번째 요소 전달
decoder(target[:, 0], hidden, cell, latent_vector)

(tensor([[[ 0.0502, -0.0365,  0.0391,  ..., -0.0059, -0.0562,  0.0437]],
 
         [[ 0.0504, -0.0361,  0.0392,  ..., -0.0064, -0.0559,  0.0437]],
 
         [[ 0.0502, -0.0364,  0.0388,  ..., -0.0060, -0.0558,  0.0435]],
 
         ...,
 
         [[ 0.0500, -0.0361,  0.0386,  ..., -0.0061, -0.0560,  0.0436]],
 
         [[ 0.0502, -0.0366,  0.0389,  ..., -0.0063, -0.0562,  0.0434]],
 
         [[ 0.0503, -0.0366,  0.0386,  ..., -0.0058, -0.0562,  0.0436]]],
        grad_fn=<ViewBackward0>),
 (tensor([[[ 0.0042, -0.0067, -0.0110,  ...,  0.0255, -0.0188, -0.0109],
           [ 0.0045, -0.0060, -0.0103,  ...,  0.0257, -0.0188, -0.0111],
           [ 0.0043, -0.0064, -0.0102,  ...,  0.0255, -0.0187, -0.0107],
           ...,
           [ 0.0047, -0.0068, -0.0106,  ...,  0.0256, -0.0186, -0.0108],
           [ 0.0042, -0.0063, -0.0102,  ...,  0.0253, -0.0194, -0.0115],
           [ 0.0044, -0.0068, -0.0101,  ...,  0.0254, -0.0185, -0.0110]]],
         grad_fn=<StackBackward0>),
  tensor(

In [230]:
# seq2seq 초기화
seq2seq = Seq2Seq(encoder, decoder)

In [246]:
# cross entropy 사용 방법에 맞게 축 변환
pred = seq2seq(source, target).permute(0, 2, 1) # (batch_size, seq_len, dim) -> (batch_size, dim, seq_len)

# cross entropy loss 계산
F.cross_entropy(pred, target)

tensor(6.9087, grad_fn=<NllLoss2DBackward0>)